In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()

llm= ChatGroq(model = "openai/gpt-oss-120b")


In [3]:
class JokeState(TypedDict):
    
    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [5]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [6]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [7]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza apply for a job?\n\nBecause it needed the *dough* and wanted to prove it could really *deliver* under pressure! 🍕😄',
 'explanation': '**Breaking Down the Joke**\n\n| Part of the joke | What it literally means | The wordplay / double‑meaning |\n|------------------|------------------------|------------------------------|\n| **“Why did the pizza apply for a job?”** | A pizza (the food) is being treated as if it were a person looking for employment. | Sets up the expectation that the punch‑line will involve a reason that makes sense for a *person* looking for work. |\n| **“Because it needed the *dough* …”** | *Dough* is the raw, uncooked bread that becomes the crust of a pizza. | *Dough* is also slang for money. So the pizza “needs the dough” in the sense of needing cash—just like a job‑seeker. |\n| **“…and wanted to prove it could really *deliver* under pressure!”** | A pizza can be *delivered* to a customer, and “under pressure” can refer to

In [8]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza apply for a job?\n\nBecause it needed the *dough* and wanted to prove it could really *deliver* under pressure! 🍕😄', 'explanation': '**Breaking Down the Joke**\n\n| Part of the joke | What it literally means | The wordplay / double‑meaning |\n|------------------|------------------------|------------------------------|\n| **“Why did the pizza apply for a job?”** | A pizza (the food) is being treated as if it were a person looking for employment. | Sets up the expectation that the punch‑line will involve a reason that makes sense for a *person* looking for work. |\n| **“Because it needed the *dough* …”** | *Dough* is the raw, uncooked bread that becomes the crust of a pizza. | *Dough* is also slang for money. So the pizza “needs the dough” in the sense of needing cash—just like a job‑seeker. |\n| **“…and wanted to prove it could really *deliver* under pressure!”** | A pizza can be *delivered* to a customer, and “under pre

In [9]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza apply for a job?\n\nBecause it needed the *dough* and wanted to prove it could really *deliver* under pressure! 🍕😄', 'explanation': '**Breaking Down the Joke**\n\n| Part of the joke | What it literally means | The wordplay / double‑meaning |\n|------------------|------------------------|------------------------------|\n| **“Why did the pizza apply for a job?”** | A pizza (the food) is being treated as if it were a person looking for employment. | Sets up the expectation that the punch‑line will involve a reason that makes sense for a *person* looking for work. |\n| **“Because it needed the *dough* …”** | *Dough* is the raw, uncooked bread that becomes the crust of a pizza. | *Dough* is also slang for money. So the pizza “needs the dough” in the sense of needing cash—just like a job‑seeker. |\n| **“…and wanted to prove it could really *deliver* under pressure!”** | A pizza can be *delivered* to a customer, and “under pr

In [10]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti get a promotion?  \n\nBecause it knew how to *pasta* the competition and always *sauced* the situation! 🍝😄',
 'explanation': '**What makes the joke funny?**  \n\n1. **The set‑up looks like a classic “Why did X get a promotion?” riddle.**  \n   In the workplace world, getting a promotion usually means you’ve out‑performed your peers, shown leadership, or brought something extra to the table. The listener expects a serious, work‑related answer.\n\n2. **The punch‑line swaps business jargon for food‑related wordplay.**  \n   - **“pasta the competition”** sounds like **“passed the competition.”**  \n     *Passed* is the verb you’d use when someone beats the competition (“She passed the competition and got the job”). By replacing *passed* with *pasta*—the Italian staple that spaghetti belongs to—the joke turns a standard achievement into a culinary gag.  \n\n   - **“sauced the situation”** mirrors the phrase **“sawed the situation”** or more

In [11]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti get a promotion?  \n\nBecause it knew how to *pasta* the competition and always *sauced* the situation! 🍝😄', 'explanation': '**What makes the joke funny?**  \n\n1. **The set‑up looks like a classic “Why did X get a promotion?” riddle.**  \n   In the workplace world, getting a promotion usually means you’ve out‑performed your peers, shown leadership, or brought something extra to the table. The listener expects a serious, work‑related answer.\n\n2. **The punch‑line swaps business jargon for food‑related wordplay.**  \n   - **“pasta the competition”** sounds like **“passed the competition.”**  \n     *Passed* is the verb you’d use when someone beats the competition (“She passed the competition and got the job”). By replacing *passed* with *pasta*—the Italian staple that spaghetti belongs to—the joke turns a standard achievement into a culinary gag.  \n\n   - **“sauced the situation”** mirrors the phrase **“sawed the s

In [12]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti get a promotion?  \n\nBecause it knew how to *pasta* the competition and always *sauced* the situation! 🍝😄', 'explanation': '**What makes the joke funny?**  \n\n1. **The set‑up looks like a classic “Why did X get a promotion?” riddle.**  \n   In the workplace world, getting a promotion usually means you’ve out‑performed your peers, shown leadership, or brought something extra to the table. The listener expects a serious, work‑related answer.\n\n2. **The punch‑line swaps business jargon for food‑related wordplay.**  \n   - **“pasta the competition”** sounds like **“passed the competition.”**  \n     *Passed* is the verb you’d use when someone beats the competition (“She passed the competition and got the job”). By replacing *passed* with *pasta*—the Italian staple that spaghetti belongs to—the joke turns a standard achievement into a culinary gag.  \n\n   - **“sauced the situation”** mirrors the phrase **“sawed the 

### TIME TRAVEL


In [13]:

workflow.get_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f10123a-7327-6315-8000-94f839925bd4"}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_id': '1f10123a-7327-6315-8000-94f839925bd4'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

### FAULT TOLERANCE

In [14]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [15]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [16]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(10)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [17]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [18]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
✅ Step 3 executed


In [19]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...

✅ Final State: {'input': 'start', 'step1': 'done', 'step2': 'done'}


In [20]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))


🔁 Re-running the graph to demonstrate fault tolerance...

✅ Final State: {'input': 'start', 'step1': 'done', 'step2': 'done'}


[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f101249-30af-6397-8003-4fe98ca770a7'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2026-02-03T17:20:02.261477+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f101249-30ac-6364-8002-939c56295dfd'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f101249-30ac-6364-8002-939c56295dfd'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-03T17:20:02.260233+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f101248-d149-6542-8001-deec26066013'}}, tasks=(PregelTask(id='92c1eb9e-4989-4fea-0a52-9f1cc3cc723